In [1]:
from astroCAST.analysis import Video, Events, Correlation
from pathlib import Path

In [2]:
folder = Path("/media/janrei1/data/method_testing/")

event_folder = folder.joinpath("22A7x9-1.roi") # your input file name here. 'h5' or 'tdb' is recommended for parallel processing 
assert event_folder.is_dir()

In [3]:
events = Events(event_folder)
events.events.columns

Index(['z0', 'z1', 'x0', 'x1', 'y0', 'y1', 'dz', 'dx', 'dy', 'area',
       'bbox_pix_num', 'mask', 'mask_centroid_local-0',
       'mask_centroid_local-1', 'mask_centroid_local-2',
       'mask_axis_major_length', 'mask_axis_minor_length', 'mask_extent',
       'mask_solidity', 'mask_area', 'mask_equivalent_diameter_area',
       'contours', 'footprint', 'fp_centroid_local-0', 'fp_centroid_local-1',
       'fp_axis_major_length', 'fp_axis_minor_length', 'fp_eccentricity',
       'fp_equivalent_diameter_area', 'fp_extent', 'fp_feret_diameter_max',
       'fp_orientation', 'fp_perimeter', 'fp_solidity', 'fp_area',
       'fp_area_convex', 'fp_cx', 'fp_cy', 'trace', 'error', 'area_norm', 'cx',
       'cy'],
      dtype='object')

In [5]:
v = events.show_event_map(video=folder.joinpath("22A7x9-1.tiff"), z_slice=(0, 10))

/opt/anaconda/envs/aip/lib/python3.9/site-packages/napari/plugins/_plugin_manager.py:542: UserWarning: Plugin 'napari-time-slicer' has already registered a function widget 'convert to 2d timelapse' which has now been overwritten
  warn(message=warn_message)


# Exploration

In [4]:
import napari
import logging 
from skimage.measure import marching_cubes, find_contours
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tifffile as tiff

In [5]:


def show_events_as_shapes(self):

        viewer = napari.Viewer()

        event_map = self.event_map.compute() # todo: very inefficient
        # event_map[event_map > 1] = 0
        # event_map = event_map[1:3, :, :]

        viewer.add_image(event_map)
        
        logging.warning(f"event_map: {event_map.shape}")
        logging.warning(f"type: {type(event_map)}")
        # logging.warning(f"event_map: {event_map[0, :5, :5]}")

        contours = marching_cubes(event_map, 0.5)
        verts, faces, normals, values = contours
        
        """
        contours = verts
        # contours = np.array([[11, 13], [111, 113], [22, 246]])
        logging.warning(f"contours: {contours}")
        logging.warning(f"contours: {contours.shape}")
        
        viewer.add_shapes(contours,
                          shape_type="polygon", edge_width=1,
                          edge_color="coral", face_color="royalblue")
        """
        
        surface = (verts, faces, values)
        viewer.add_surface(surface)
        
        
# show_events_as_shapes(events)

In [6]:
def get_contour(mask):
    
    mask = np.pad(mask, pad_width=((1, 1), (1, 1), (1, 1)), mode="constant", constant_values=0)
    
    contours = []
    for cz in range(1, mask.shape[0]-1):
        
        frame = mask[cz, :, :]
        
        # find countours in frame
        contour = find_contours(frame, level=0.9)[0]
            
        # create z axis
        z_column = np.zeros((contour.shape[0], 1))
        z_column[:] = cz
        
        # add extra dimension for z axis
        contour = np.append(z_column, contour, axis=1)
                
        # adjust for padding
        contour = np.subtract(contour, 1)
            
        contours.append(contour)
        
    return contours

# ev = events.events.iloc[0]
# mask = np.reshape(ev["mask"], newshape=(ev.dz, ev.dx, ev.dy))

# contours = get_contour(mask)
# # contours[1]

In [7]:
# contours

In [8]:
# display(contours[0][0])

# plt.imshow(mask[0, :, :])

In [9]:
# viewer = napari.Viewer()

# viewer.add_image(mask)
# viewer.add_shapes(contours, shape_type="polygon")


In [37]:
viewer = napari.Viewer()

max_id = 52

# em = events.event_map.copy()
# em[em > max_id] = 0

em = tiff.imread(folder.joinpath("22A7x9-1.tiff"))

viewer.add_image(em)

ev_sel = events.events.head(max_id)

contours_adjusted = []
for _, ev in tqdm(ev_sel.iterrows(), total=len(ev_sel)):
    
    contours = ev.contours#.copy()
    
    if len(contours) < 1:
        continue
    
    for contour in contours:
        
        c = np.zeros(contour.shape)
        c[:] = contour[:] # todo: no idea why we need this but otherwise the coordinates keep getting added
                        
        c[:, 1] += ev.x0
        c[:, 2] += ev.y0
                
        contours_adjusted.append(c)
    
    
viewer.add_shapes(contours_adjusted, shape_type="polygon", edge_color="red", face_color="#80808000")   
    


100%|█████████████████████████████████████████| 52/52 [00:00<00:00, 2624.15it/s]


<Shapes layer 'contours_adjusted' at 0x7f34e6d33f40>

In [7]:
viewer = napari.Viewer()

max_id = 52

em = events.event_map.copy()
# em[em > max_id] = 0

dff = tiff.imread(folder.joinpath("22A7x9-1.tiff"))

viewer.add_image(dff, name="dff")
viewer.add_labels(em, name="roi")


/opt/anaconda/envs/aip/lib/python3.9/site-packages/napari/plugins/_plugin_manager.py:542: UserWarning: Plugin 'napari-time-slicer' has already registered a function widget 'convert to 2d timelapse' which has now been overwritten
  warn(message=warn_message)


<Labels layer 'roi' at 0x7f4e8923ea00>

In [11]:
## show overlay of events (event_map)
events.show_events()


AttributeError: 'Events' object has no attribute 'show_events'

In [ ]:
## summary table

In [ ]:
## distribution plots

In [ ]:
## filtering

# Correlation

# Comparison